# Seminario 5. Expresiones regulares en Python.

Lo que llamamos **expresiones regulares** son *especificaciones formales de patrones de texto*. Casi todos los lenguajes de programación incorporan algún mecanismo para definirlas y aplicarlas en la búsqueda de subcadenas de texto que encajen en un determinado patrón o bien para modificar de alguna manera las subcadenas que encajen con el patrón. La sintaxis de las expresiones regulares incluye reglas para especificar todo tipo de eventos: texto literal, repeticiones (finitas o infinitas), combinaciones de patrones, etc. Se utilizan profusamente en todo tipo de aplicaciones que requieren procesar texto (editores, IDEs, procesadores de comandos, etc.). En Python existe un módulo de la biblioteca standard especializado en expresiones regulares: el módulo `re`.

> Aunque la definición formal de *expresión regular* se limita a expresiones que describen lenguajes regulares, la sintaxis del módulo `re` permite definir conjuntos de expresiones mucho más generales. En este seminario, utilizaremos el término *expresión regular* en un sentido amplio para referirnos a cualquier expresión que pueda ser evaluada por el módulo `re` de Python.

## Búsqueda de patrones en texto.

El uso más habitual del módulo `re` es la búsqueda de patrones en una cadena de caracteres. La función `search()` toma como argumentos un patrón y una cadena de caracteres a explorar, y retorna un objeto de la clase `Match` si el patrón encaja con alguna subcadena o `None` si no encaja con nada. El objeto de la clase `Match` incluye información sobre la expresión regular, la cadena explorada y la localización del patrón dentro de ésta. Veámoslo con un ejemplo:

In [1]:
import re

pat = 'este'
t = '¿Encaja este texto con el patrón?'

match = re.search(pat, t)

if match is not None:
    s = match.start() # posición de inicio del patrón en la cadena t
    e = match.end()   # posición siguiente al final del patrón en la cadena t

    print(f"Patrón {match.re.pattern!r} encontrado en {t!r}:")
    print(f"\tPosición inicial: {s}")
    print(f"\tPosición final: {e-1}")
    print(f"\tSubcadena encontrada: {t[s:e]!r}")
else:
    print(f"Patrón {pat!r} NO encontrado en {t!r}")

Patrón 'este' encontrado en '¿Encaja este texto con el patrón?':
	Posición inicial: 8
	Posición final: 11
	Subcadena encontrada: 'este'


## Compilación de expresiones regulares.

El módulo `re` dispone de métodos que permiten especificar las expresiones regulares en forma de cadenas de caracteres. Sin embargo, si van a a usarse con frecuencia, es más eficiente compilarlas, es decir, transformarlas en autómatas, lo cual acelera enormemente el procesado. Precisamente, la función `compile()` convierte una expresión regular en forma de cadena de caracteres en un objeto de la clase `RegexObject`:

In [2]:
import re

# Compilación de patrones
regexes = [re.compile(p) for p in ('esto', 'esta', 'este')]

t = '¿Encaja este texto con el patrón?'
print(f"Texto: {t!r}")

for r in regexes:
    print(f"Búsqueda de {r.pattern!r} -->", end=' ')
    if r.search(t):
        print("Encontrado")
    else:
        print("No encontrado")

Texto: '¿Encaja este texto con el patrón?'
Búsqueda de 'esto' --> No encontrado
Búsqueda de 'esta' --> No encontrado
Búsqueda de 'este' --> Encontrado


Al utilizar expresiones compiladas, el esfuerzo de compilación se concentra en la inicialización de un programa, de forma que cada búsqueda se realiza en un tiempo considerablemente menor.

## Búsqueda de múltiples instancias.

Hasta ahora, los ejemplos han mostrado casos en los que se reportaba la localización de una única instancia del patrón buscado. La función `findall()` retorna todas las subcadenas dentro el texto que, sin solaparse, encajan con el patrón suministrado. Por ejemplo:

In [3]:
import re

t = 'abbaaaabbbbaaaa'
pat = 'ab+' # una a seguida de una o más b's

for i, match in enumerate(re.findall(pat, t)):
    print(f"Instancia {i+1}: {match!r}")

Instancia 1: 'abb'
Instancia 2: 'abbbb'


Si no nos basta con las subcadenas y deseamos obtener toda la información sobre cada instancia que encaja con el patrón, podemos utilizar la función `finditer()`, que retorna un iterador que recorre todos los objetos de la clase `Match` que encajan con el patrón:

In [4]:
import re

t = 'abbaaaabbbbaaaa'
pat = 'ab+' # una a seguida de una o más b's

for i, match in enumerate(re.finditer(pat, t)):
    s = match.start()
    e = match.end()
    print(f"Instancia {i+1}: {t[s:e]!r} en {s}:{e-1}")

Instancia 1: 'abb' en 0:2
Instancia 2: 'abbbb' en 6:10


## Sintaxis de las expresiones regulares.

Como hemos visto en el ejemplo anterior, las expresiones regulares permiten detectar patrones sofisticados, no simples literales. Los patrones pueden incluir repeticiones (como en 'ab+'), pueden estar ligados a determinadas partes de la entrada (el principio o el final, por ejemplo) y, en general, pueden expresarse de manera compacta, sin que haga falta especificar cada carácter o todos los posibles literales (que podrían ser infinitos). Para ello, existen los llamados *meta-caracteres* (como el símbolo '+' en el ejemplo anterior). En primer lugar, definiremos una función auxiliar:

In [5]:
import re

def test_pat(t, l_pat):
    """ Dado un texto t y una lista de patrones l_pat, busca en el texto
        instancias de cada patrón y los muestra por la consola
        Cada elemento de l_pat es una tupla con un patrón y su descripción
    """
    for pat, desc in l_pat:
        print(f"'{pat}' ({desc})\n")
        print(f"\t{t!r}")
        for match in re.finditer(pat, t):
            s = match.start()
            e = match.end()
            substr = t[s:e]
            prefix = ' ' * s
            print(f"\t{prefix}{substr!r}")
        print()
    return

# Ejemplo de aplicación
if __name__ == '__main__':
    texto = 'abbaaabbbbaaaaa'
    lista_patrones = [('ab+', "'a' seguida de una o más 'b'"),
                      ('ba+', "'b' seguida de una o más 'a'")]
    test_pat(texto, lista_patrones)

'ab+' ('a' seguida de una o más 'b')

	'abbaaabbbbaaaaa'
	'abb'
	     'abbbb'

'ba+' ('b' seguida de una o más 'a')

	'abbaaabbbbaaaaa'
	  'baaa'
	         'baaaaa'



### Repeticiones.

Hay 5 formas de expresar la repetición de elementos dentro de un patrón:

- `*`: Un patrón seguido del metacarácter `*` quiere decir que puede aparecer cero o más veces
- `+`: Un patrón seguido del metacarácter `+` quiere decir que puede aparecer una o más veces
- `?`: Un patrón seguido del metacarácter `?` quiere decir que puede aparecer una vez o ninguna
- `{m}`: Un patrón seguido de la secuencia `{m}` quiere decir que tiene que aparecer exactamente `m` veces
- `{m,n}`: Un patrón seguido de la secuencia `{m,n}` quiere decir que puede aparecer entre `m` y `n` veces (m<n); si no se suministra el segundo elemento (es decir, dando la secuencia `{m,}`), quiere decir que el patrón tiene que aparecer `m` o más veces

Veamos algunos ejemplos:

In [6]:
test_pat('abbaabbbba',
         [('ab*', 'a seguida de cero o más b'),
          ('ab+', 'a seguida de una o más b'),
          ('ab?', 'a seguida de una o cero b'),
          ('ab{3}', 'a seguida de tres b'),
          ('ab{2,4}', 'a seguida de entre dos y cuatro b')],
)

'ab*' (a seguida de cero o más b)

	'abbaabbbba'
	'abb'
	   'a'
	    'abbbb'
	         'a'

'ab+' (a seguida de una o más b)

	'abbaabbbba'
	'abb'
	    'abbbb'

'ab?' (a seguida de una o cero b)

	'abbaabbbba'
	'ab'
	   'a'
	    'ab'
	         'a'

'ab{3}' (a seguida de tres b)

	'abbaabbbba'
	    'abbb'

'ab{2,4}' (a seguida de entre dos y cuatro b)

	'abbaabbbba'
	'abb'
	    'abbbb'



Obsérvese que, al procesar una repetición, el módulo `re` consume tantos símbolos como sea posible de forma que encaje el patrón correspondiente. Este comportamiento **voraz** puede resultar en un número pequeño de instancias detectadas o en que las instancias detectadas contengan más texto del que se pretendía. El comportamiento voraz puede desactivarse mediante el metacarácter `?` a continuación de la instrucción de repetición. Esto hace que cada coincidencia contenga el *mínimo número de símbolos* para que encaje el patrón correspondiente. Veamos cómo cambia la salida al desactivar el comportamiento voraz de las repeticiones en el ejemplo anterior:

In [7]:
test_pat('abbaabbbba',
         [('ab*?', 'a seguida de cero o más b'),
          ('ab+?', 'a seguida de una o más b'),
          ('ab??', 'a seguida de una o cero b'),
          ('ab{3}?', 'a seguida de tres b'),
          ('ab{2,4}?', 'a seguida de entre dos y cuatro b')],
)

'ab*?' (a seguida de cero o más b)

	'abbaabbbba'
	'a'
	   'a'
	    'a'
	         'a'

'ab+?' (a seguida de una o más b)

	'abbaabbbba'
	'ab'
	    'ab'

'ab??' (a seguida de una o cero b)

	'abbaabbbba'
	'a'
	   'a'
	    'a'
	         'a'

'ab{3}?' (a seguida de tres b)

	'abbaabbbba'
	    'abbb'

'ab{2,4}?' (a seguida de entre dos y cuatro b)

	'abbaabbbba'
	'abb'
	    'abb'



Nótese cómo, al desactivar el comportamiento voraz, los patrones que permiten cero ocurrencias de la 'b' al final ('ab*?', 'ab??') detectan instancias que no contienen ninguna 'b' al final.

### Conjuntos de caracteres.

Un *conjunto de caracteres* se utiliza cuando se desea que cualquiera de dichos caracteres sea admitido en una determinada posición de un patrón. Los conjuntos de caracteres se definen mediante el uso de corchetes. Por ejemplo, la expresión `[abc]` encaja con cualquiera de los tres caracteres consignados entre corchetes. Veámoslo con un ejemplo:

In [8]:
test_pat('abbaabbba',
         [('[ab]', 'una a o una b'),
          ('a[ab]+', 'una a seguida de una o más ocurrencias de a o b'),
          ('a[ab]+?', 'una a seguida de una o más ocurrencias de a o b, no voraz')])

'[ab]' (una a o una b)

	'abbaabbba'
	'a'
	 'b'
	  'b'
	   'a'
	    'a'
	     'b'
	      'b'
	       'b'
	        'a'

'a[ab]+' (una a seguida de una o más ocurrencias de a o b)

	'abbaabbba'
	'abbaabbba'

'a[ab]+?' (una a seguida de una o más ocurrencias de a o b, no voraz)

	'abbaabbba'
	'ab'
	   'aa'



Los conjuntos de caracteres también pueden utilizarse para *excluir* determinados caracteres en una posición del patrón, es decir, para *admitir cualquier carácter salvo los consignados en el conjunto*. Esto se consigue colocando un acento circunflejo (`^`) al principio del conjunto, es decir, mediante una expresión de la forma `[^ ... ]`. En el siguiente ejemplo, tratamos de detectar cualquier secuencia de caracteres que no contenga ni espacios en blanco, ni puntos ni comas:

In [9]:
test_pat('Esto, por exigencias de guión, es un texto con puntuación.',
         [('[^,. ]+', 'secuencia sin puntos, comas ni espacios en blanco')])

'[^,. ]+' (secuencia sin puntos, comas ni espacios en blanco)

	'Esto, por exigencias de guión, es un texto con puntuación.'
	'Esto'
	      'por'
	          'exigencias'
	                     'de'
	                        'guión'
	                               'es'
	                                  'un'
	                                     'texto'
	                                           'con'
	                                               'puntuación'



Es posible especificar *rangos de caracteres* para así considerar todos los caracteres incluidos entre los caracteres primero y último de dicho rango. En el siguiente ejemplo, los rangos `a-z` y `A-Z` se refieren a las letras minúsculas y mayúsculas de la tabla ASCII, respectivamente. Nótese que es posible combinar distintos rangos con caracteres sencillos dentro de un mismo conjunto de caracteres:

In [10]:
test_pat('Esto, por exigencias de guión, es un texto con puntuación.',
         [('[a-záéíóúü]+', 'secuencia de letras minúsculas'),
          ('[A-ZÁÉÍÓÚÜ]+', 'secuencia de letras mayúsculas'),
          ('[a-záéíóúüA-ZÁÉÍÓÚÜ]+', 'secuencia de letras (mayúsculas o minúsculas)'),
          ('[A-ZÁÉÍÓÚÜ][a-záéíóúü]+', 'una letra mayúscula seguida por una o más letras minúsculas')])

'[a-záéíóúü]+' (secuencia de letras minúsculas)

	'Esto, por exigencias de guión, es un texto con puntuación.'
	 'sto'
	      'por'
	          'exigencias'
	                     'de'
	                        'guión'
	                               'es'
	                                  'un'
	                                     'texto'
	                                           'con'
	                                               'puntuación'

'[A-ZÁÉÍÓÚÜ]+' (secuencia de letras mayúsculas)

	'Esto, por exigencias de guión, es un texto con puntuación.'
	'E'

'[a-záéíóúüA-ZÁÉÍÓÚÜ]+' (secuencia de letras (mayúsculas o minúsculas))

	'Esto, por exigencias de guión, es un texto con puntuación.'
	'Esto'
	      'por'
	          'exigencias'
	                     'de'
	                        'guión'
	                               'es'
	                                  'un'
	                                     'texto'
	                                           'con'
	                  

Como un caso especial de conjunto de caracteres, el metacarácter punto (`.`) representa una sola aparición de cualquier carácter. En combinación con los metacaracteres de repetición, puede resultar muy útil, con o sin procesamiento voraz.

In [11]:
test_pat('abbaabbba',
         [('a.', 'a seguida de un carácter cualquiera'),
          ('b.', 'b seguida de un carácter cualquiera'),
          ('a.*b', 'a seguida de cualquier secuencia de caracteres acabada en b, lo más larga posible'),
          ('a.*?b', 'a seguida de cualquier secuencia de caracteres acabada en b, lo más corta posible')])

'a.' (a seguida de un carácter cualquiera)

	'abbaabbba'
	'ab'
	   'aa'

'b.' (b seguida de un carácter cualquiera)

	'abbaabbba'
	 'bb'
	     'bb'
	       'ba'

'a.*b' (a seguida de cualquier secuencia de caracteres acabada en b, lo más larga posible)

	'abbaabbba'
	'abbaabbb'

'a.*?b' (a seguida de cualquier secuencia de caracteres acabada en b, lo más corta posible)

	'abbaabbba'
	'ab'
	   'aab'



### Conjuntos predefinidos de caracteres.

En la tabla siguiente se muestran los códigos de escape (letras precedidas del símbolo *backslash*: `\`) reconocidos por el módulo `re`, que representan distintos conjuntos de caracteres:

Código | Significado
:-- | :--
\\d | Dígitos
\\D | No dígitos
\\s | Espacios (tabulador, espacio, salto de línea, etc.)
\\S | No espacios
\\w | Caracteres alfanuméricos
\\W | Caracteres no alfanuméricos

Los símbolos *backslash* (`\`) deben ser precedidos por otro backslash para ser reconocidos como tales en Python. Para evitarlo, se recomienda usar las llamadas *raw strings*, poniendo la letra `r` por delante de la propia cadena. Así, por ejemplo, `r'\d+'` representa una cadena formada por uno o más dígitos. Veamos algunos ejemplos:

In [12]:
test_pat('(Un primer ejemplo) - Ejemplo #1:',
         [(r'\d+', 'una secuencia de dígitos'),
          (r'\D+', 'una secuencia de no dígitos'),
          (r'\s+', 'una secuencia de espacios'),
          (r'\S+', 'una secuencia de no espacios'),
          (r'\w+', 'una secuencia de caracteres alfanuméricos'),
          (r'\W+', 'una secuencia de caracteres no alfanuméricos')])

'\d+' (una secuencia de dígitos)

	'(Un primer ejemplo) - Ejemplo #1:'
	                               '1'

'\D+' (una secuencia de no dígitos)

	'(Un primer ejemplo) - Ejemplo #1:'
	'(Un primer ejemplo) - Ejemplo #'
	                                ':'

'\s+' (una secuencia de espacios)

	'(Un primer ejemplo) - Ejemplo #1:'
	   ' '
	          ' '
	                   ' '
	                     ' '
	                             ' '

'\S+' (una secuencia de no espacios)

	'(Un primer ejemplo) - Ejemplo #1:'
	'(Un'
	    'primer'
	           'ejemplo)'
	                    '-'
	                      'Ejemplo'
	                              '#1:'

'\w+' (una secuencia de caracteres alfanuméricos)

	'(Un primer ejemplo) - Ejemplo #1:'
	 'Un'
	    'primer'
	           'ejemplo'
	                      'Ejemplo'
	                               '1'

'\W+' (una secuencia de caracteres no alfanuméricos)

	'(Un primer ejemplo) - Ejemplo #1:'
	'('
	   ' '
	          ' '
	                  ') - '
	   

### Localización relativa de patrones.

Además de describir el contenido de un patrón, es posible especificar también la localización relativa donde se desea encontrar dicho patrón (al principio de una palabra, al final de una línea, etc.). A continuación se muestran los códigos de localización reconocidos por el módulo `re`:

Código | Significado
:-- | :--
^ | Inicio de cadena o de línea
$ | Final de cadena o de línea
\\A | Inicio de cadena
\\Z | Final de cadena
\\b | Cadena vacía situada al inicio o final de una palabra
\\B | Cadena vacía no situada al inicio o final de una palabra

En el ejemplo siguiente, los patrones especificados para detectar palabras al inicio o al final de una cadena son diferentes porque una palabra que aparezca al final de la cadena irá seguida de un signo de puntuación. Si utilizáramos el patrón `r'\w+$'`, no conseguiríamos detectar dicha palabra, porque el signo de puntuación final no es un carácter alfanumérico.

In [13]:
test_pat('Esto, por exigencias de guión, es un texto con puntuación.',
         [(r'^\w+', 'palabra al inicio de la cadena'),
          (r'\A\w+', 'palabra al inicio de la cadena'),
          (r'\w+\S*$', 'palabra próxima al final de la cadena'),
          (r'\w+\S*\Z', 'palabra próxima al final de la cadena'),
          (r'\w*t\w*', 'palabra que contiene la letra t'),
          (r'\bt\w+', 'letra t al inicio de una palabra'),
          (r'\w+s\b', 'letra s al final de una palabra'),
          (r'\Bn\B', 'letra n, ni al principio ni al final de una palabra')],
)

'^\w+' (palabra al inicio de la cadena)

	'Esto, por exigencias de guión, es un texto con puntuación.'
	'Esto'

'\A\w+' (palabra al inicio de la cadena)

	'Esto, por exigencias de guión, es un texto con puntuación.'
	'Esto'

'\w+\S*$' (palabra próxima al final de la cadena)

	'Esto, por exigencias de guión, es un texto con puntuación.'
	                                               'puntuación.'

'\w+\S*\Z' (palabra próxima al final de la cadena)

	'Esto, por exigencias de guión, es un texto con puntuación.'
	                                               'puntuación.'

'\w*t\w*' (palabra que contiene la letra t)

	'Esto, por exigencias de guión, es un texto con puntuación.'
	'Esto'
	                                     'texto'
	                                               'puntuación'

'\bt\w+' (letra t al inicio de una palabra)

	'Esto, por exigencias de guión, es un texto con puntuación.'
	                                     'texto'

'\w+s\b' (letra s al final de una palabra)

	

## Búsqueda de patrones restringida

A veces interesa encontrar instancias de los patrones sólo en regiones determinadas del texto. Para conseguirlo podemos indicar al módulo `re` que limite la búsqueda a dichas regiones. Por ejempo, si el patrón debe aparecer al inicio de la cadena, utilizar el método `match()` en lugar de `search()` localizará la búsqueda sin necesidad de incluir dicha localización en el patrón:

In [14]:
t = 'Esto, por exigencias de guión, es un texto con puntuación.'
pat = 'es'

print(f'Texto: {t!r}')
print(f'Patrón: {pat!r}')

m = re.match(pat, t)
print(f'Resultado de la búsqueda con match(): {m}')

s = re.search(pat, t)
print(f'Resultado de la búsqueda con search(): {s}')

Texto: 'Esto, por exigencias de guión, es un texto con puntuación.'
Patrón: 'es'
Resultado de la búsqueda con match(): None
Resultado de la búsqueda con search(): <re.Match object; span=(31, 33), match='es'>


Como el texto literal 'es' no aparece al inicio de la cadena, el método `match()` no lo encuentra. Sin embargo, el método `search()` recorre toda la cadena para buscar instancias del patrón y localiza la cadena 'es' en la posición 31.

En el caso del método `fullmatch()`, la cadena completa debe encajar con el patrón. En el siguiente ejemplo, la llamada a `search()` demuestra que el patrón 'es' aparece dentro de la cadena pero, como no la abarca por completo, el método `fullmatch()` retorna `None`:

In [15]:
t = 'Esto, por exigencias de guión, es un texto con puntuación.'
pat = 'es'

print(f'Texto: {t!r}')
print(f'Patrón: {pat!r}')

s = re.search(pat, t)
print(f'Resultado de la búsqueda con search(): {s}')

m = re.fullmatch(pat, t)
print(f'Resultado de la búsqueda con fullmatch(): {m}')

Texto: 'Esto, por exigencias de guión, es un texto con puntuación.'
Patrón: 'es'
Resultado de la búsqueda con search(): <re.Match object; span=(31, 33), match='es'>
Resultado de la búsqueda con fullmatch(): None


El método `search()`, aplicado a una expresión regular compilada, admite los argumentos `start` y `end` que limitan la búsqueda al tramo correspondinete de la cadena. En el siguiente ejemplo se implementa una versión poco eficiente del método `iterall()`, que repite iterativamente la búsqueda a partir del final de la última instancia del patrón encontrada, hasta llegar al final de la cadena:

In [16]:
t = 'Dime, de verdad, qué sabes de ella.'
pat = re.compile(r'\b\w*de\w*\b') # detecta la palabra 'de'

print(f'Texto: {t!r}\n')

pos = 0
while True:
    match = pat.search(t, pos)
    if not match:
        break
    s = match.start()
    e = match.end()
    print(f'  {s:>2d} : {e-1:>2d} = {t[s:e]!r}')
    pos = e # posición de inicio de la siguiente búsqueda

Texto: 'Dime, de verdad, qué sabes de ella.'

   6 :  7 = 'de'
  27 : 28 = 'de'


## Agrupamiento de patrones.

La potencia de las expresiones regulares proviene de su capacidad para encontrar instancias de patrones en una cadena. Agrupando partes de los patrones es posible realizar análisis sofisticados de las cadenas de entrada para ver si cumplen determinadas especificaciones. Esto es lo que se conoce como **parsing**. Para que el módulo `re` reconozca un grupo, basta poner un patrón entre paréntesis: `(patrón)`. Cualquier expresión regular puede constituir un grupo sin más que ponerla entre paréntesis, lo cual permite utilizarla como parte de una expresión regular más grande. Nótese que podemos aplicar los operadores de repetición a un grupo, y de esa manera detectar cualquier sucesión de símbolos que repita el patrón especificado entre paréntesis. Veamos unos ejemplos:

In [17]:
test_pat('abbaaabbbbaaaaa',
         [('a(ab)', 'a seguida del literal "ab"'),
          ('a(a*b*)', 'a seguida de 0-n repeticiones de a y 0-n repeticiones de b'),
          ('a(ab)*', 'a seguida de 0-n repeticiones del literal "ab"'),
          ('a(ab)+', 'a seguida de 1-n repeticiones del literal "ab"')])

'a(ab)' (a seguida del literal "ab")

	'abbaaabbbbaaaaa'
	    'aab'

'a(a*b*)' (a seguida de 0-n repeticiones de a y 0-n repeticiones de b)

	'abbaaabbbbaaaaa'
	'abb'
	   'aaabbbb'
	          'aaaaa'

'a(ab)*' (a seguida de 0-n repeticiones del literal "ab")

	'abbaaabbbbaaaaa'
	'a'
	   'a'
	    'aab'
	          'a'
	           'a'
	            'a'
	             'a'
	              'a'

'a(ab)+' (a seguida de 1-n repeticiones del literal "ab")

	'abbaaabbbbaaaaa'
	    'aab'



El método `groups()` aplicado al objeto de la clase `Match` retornado por `search()` retorna la secuencia de subcadenas que encajan con los grupos definidos dentro del patrón, en el mismo orden en que son detectadas. Veamos un ejemplo:

In [18]:
t = 'Dime, de verdad, qué sabes de ella.'

print(f'Texto: {t!r}\n')

pat = [(r'^(\w+)', 'palabra al inicio de la cadena'),
       (r'(\w+)\S*$', 'palabra al final de la cadena, opcionalmente con puntuación'),
       (r'(\bd\w+)\W+(\w+)', 'palabra empezando por d, seguida de otra palabra'),
       (r'(\w+s)\b', 'palabra que termina en s')]

for p, desc in pat:
    regex = re.compile(p)
    match = regex.search(t)
    print(f"'{p}' ({desc})\n")
    print(f'\t{match.groups()}\n')

Texto: 'Dime, de verdad, qué sabes de ella.'

'^(\w+)' (palabra al inicio de la cadena)

	('Dime',)

'(\w+)\S*$' (palabra al final de la cadena, opcionalmente con puntuación)

	('ella',)

'(\bd\w+)\W+(\w+)' (palabra empezando por d, seguida de otra palabra)

	('de', 'verdad')

'(\w+s)\b' (palabra que termina en s)

	('sabes',)



El método `group()` permite encontrar la subcadena que encaja con un grupo en particular. Esto puede ser útil cuando no todas las subcadenas que encajan con los grupos son relevantes en la búsqueda; esto sucede, por ejemplo, cuando algunos de los grupos definen el contexto y solo uno de los grupos es el objetivo real de la búsqueda. Veamos un ejemplo:

In [19]:
t = 'Dime, de verdad, qué sabes de ella.'

print(f'Texto: {t!r}\n')

# Palabra que empieza por 'd', seguida de otra palabra
regex = re.compile(r'(\bd\w+)\W+(\w+)')
print(f'Patrón:  {regex.pattern}')

match = regex.search(t)

print(f'Detección completa: {match.group(0)}')
print(f'Palabra que empieza por "d": {match.group(1)}')
print(f'Palabra que va después: {match.group(2)}')

Texto: 'Dime, de verdad, qué sabes de ella.'

Patrón:  (\bd\w+)\W+(\w+)
Detección completa: de verdad
Palabra que empieza por "d": de
Palabra que va después: verdad


Python permite dar nombre a los grupos, lo cual facilita el acceso posterior a dichos grupos y permite modificarlos sin necesidad de tocar el código que los utiliza. La sintaxis es como sigue: `(?P<nombre>patrón)`. El método `groupdict()` permite recuperar en un diccionario las subcadeanas que encajan con cada nombre de grupo. Veámoslo con un ejemplo:

In [20]:
t = 'Dime, de verdad, qué sabes de ella.'

print(f'Texto: {t!r}\n')

pat = [ r'^(?P<first_word>\w+)',
        r'(?P<last_word>\w+)\S*$',
        r'(?P<d_word>\bd\w+)\W+(?P<other_word>\w+)',
        r'(?P<ends_with_s>\w+s)\b']

for p in pat:
    regex = re.compile(p)
    match = regex.search(t)
    print(f"'{p}'")
    print(f'  {match.groups()}')
    print(f'  {match.groupdict()}\n')

Texto: 'Dime, de verdad, qué sabes de ella.'

'^(?P<first_word>\w+)'
  ('Dime',)
  {'first_word': 'Dime'}

'(?P<last_word>\w+)\S*$'
  ('ella',)
  {'last_word': 'ella'}

'(?P<d_word>\bd\w+)\W+(?P<other_word>\w+)'
  ('de', 'verdad')
  {'d_word': 'de', 'other_word': 'verdad'}

'(?P<ends_with_s>\w+s)\b'
  ('sabes',)
  {'ends_with_s': 'sabes'}



A continuación se muestra una nueva función `test_pat2()`, basada en `test_pat()`, pero que, para cada instancia encontrada del patrón, añade las subcadenas que encajan con cada uno de los grupos de dicho patrón, en orden de aparición. Con esta función, los ejemplos que mostraremos después serán más fáciles de seguir.

In [21]:
def test_pat2(t, l_pat):
    """ Dado un texto t y una lista de patrones l_pat, busca en el texto
        instancias de cada patrón y los muestra por la consola
        Cada elemento de l_pat es una tupla con un patrón y su descripción
    """
    for pat, desc in l_pat:
        print(f"'{pat}' ({desc})\n")
        print(f"\t{t!r}")
        for match in re.finditer(pat, t):
            s = match.start()
            e = match.end()
            prefix = ' ' * s
            print(f"\t{prefix}{t[s:e]!r}{' '*(len(t)-e)}", end=' ')
            print(match.groups())
            if match.groupdict():
                print(f'{" "*(len(t)-s)}{match.groupdict()}')
        print()
    return

Puesto que un grupo es en sí mismo una expresión regular, unos grupos pueden anidarse dentro de otros para construir expresiones realmente complejas. En el siguiente ejemplo, vemos lo que sucede al anidar unos grupos dentro de otros (nótese que al recorrer los grupos para mostrar las subcadenas asociadas, primero aparece el grupo más general y después los grupos anidados dentro del primero, de izquierda a derecha):

In [22]:
test_pat2('abbaabbba',
          [(r'a((a*)(b*))', 'a seguida de 0-n repeticiones de a y 0-n repeticiones de b')])

'a((a*)(b*))' (a seguida de 0-n repeticiones de a y 0-n repeticiones de b)

	'abbaabbba'
	'abb'       ('bb', '', 'bb')
	   'aabbb'  ('abbb', 'a', 'bbb')
	        'a' ('', '', '')



Los grupos son útiles también para especificar una serie de patrones alternativos, mediante el metacarácter `|`, que indica que se busca una subcadena que encaje con cualquiera de los patrones enumerados. Sin embargo, hay que utilizarlo con cuidado. El siguiente ejemplo es ilustrativo:

In [23]:
test_pat2('abbaabbba',
          [(r'a((a+)|(b+))', 'a seguida de una secuencia de 1-n a o una secuencia de 1-n b'),
           (r'a((a|b)+)', 'a seguida de una secuencia de 1-n a o b')])

'a((a+)|(b+))' (a seguida de una secuencia de 1-n a o una secuencia de 1-n b)

	'abbaabbba'
	'abb'       ('bb', None, 'bb')
	   'aa'     ('a', 'a', None)

'a((a|b)+)' (a seguida de una secuencia de 1-n a o b)

	'abbaabbba'
	'abbaabbba' ('bbaabbba', 'a')



Cuando uno de los grupos alternativos no tiene subcadena asignada, pero el patrón completo sí, el método `groups()` le asigna el valor `None`. Por otra parte, en el último caso vemos la subcadena 'bbaabbba' asignada al grupo general '((a|b)+)' y a continuación la subcadena 'a' asignada al grupo '(a|b)' en último lugar.

Derfinir un grupo con un sub-patrón puede ser útil incluso cuando la subcadena que encaja con dicho subpatrón no interesa que forme parte del resultado de la búsqueda. Este tipo de grupos se dicen *no capturados* (**non-capturing groups**) y suelen utilizarse para describir patrones repetitivos o alternativos dentro de un patrón más general. La sintaxis de un grupo no capturado es como sigue: `(?:patrón)`. Veamos un ejemplo:

In [24]:
test_pat2('abbaabbba',
          [(r'a((a+)|(b+))', 'grupos anidados que se capturan'),
          (r'a((?:a+)|(?:b+))', 'grupos anidados que no se capturan')])

'a((a+)|(b+))' (grupos anidados que se capturan)

	'abbaabbba'
	'abb'       ('bb', None, 'bb')
	   'aa'     ('a', 'a', None)

'a((?:a+)|(?:b+))' (grupos anidados que no se capturan)

	'abbaabbba'
	'abb'       ('bb',)
	   'aa'     ('a',)



Nótese que el método `groups()` sólo retorna las subcadenas asociadas a los grupos capturados. En el último caso, solo vemos la subcadena asociada al grupo general (capturado), el cual contiene dos grupos anidados no capturados.

## Opciones de búsqueda

Los métodos `compile()`, `search()` y `match()`, además de otros métodos del módulo `re` que también aceptan patrones de búsqueda, pueden modificar su comportamiento por medio de unos parámetros especiales, conocidos como *flags*, que representan opciones de búsqueda  y pueden combinarse a través del operador OR a nivel de bit. A continuación se presentan algunas de estas opciones.

### Búsqueda independiente del tamaño del carácter: minúscula o mayúscula.

El *flag* `re.IGNORECASE` hace que los caracteres literales y los rangos de caracteres que impliquen letras admitan tanto letras mayúsculas como minúsculas. Veamos un ejemplo:

In [25]:
t = 'Esto, por exigencias de guión, es un texto con puntuación.'
pat = r'\bE\w+'

with_case = re.compile(pat)
without_case = re.compile(pat, re.IGNORECASE)

print(f'Texto: {t!r}')
print(f"Patrón: '{pat}'")

print('\nTeniendo en cuenta el tamaño del carácter:')
for match in with_case.findall(t):
    print(f'  {match!r}')

print('\nSin tener en cuenta el tamaño del carácter:')
for match in without_case.findall(t):
    print(f'  {match!r}')

Texto: 'Esto, por exigencias de guión, es un texto con puntuación.'
Patrón: '\bE\w+'

Teniendo en cuenta el tamaño del carácter:
  'Esto'

Sin tener en cuenta el tamaño del carácter:
  'Esto'
  'exigencias'
  'es'


### Entrada de texto con múltiples líneas.

Hay dos *flags* que afectan a la forma en que son tratados los textos que incluyen múltiples líneas: `re.MULTILINE` y `re.DOTALL`. El primero controla cómo se procesan las instrucciones de localización en cadenas de caracteres que contienen saltos de línea (`\n`). El flag `re.MULTILINE` activa el modo multilínea, bajo el cual las reglas de localización para `^` y `$` se aplican no sólo al inicio y final de la cadena, sino también al inicio y final de cada línea, respectivamente. Veamos un ejemplo:

In [26]:
t = 'Este texto ocupa la primera línea.\nPero hay una segunda.\nY una tercera.'
pat = r'(^\w+)|(\w+\S*$)'

single_line = re.compile(pat)
multiline = re.compile(pat, re.MULTILINE)

print(f'Texto: {t!r}')
print(f"Patrón: '{pat}'")

print('\nTratando la cadena completa como una sola línea:')
for match in single_line.findall(t):
    print(f'  {match!r}')

print('\nConsiderando las líneas definidas por los saltos de línea:')
for match in multiline.findall(t):
    print(f'  {match!r}')

Texto: 'Este texto ocupa la primera línea.\nPero hay una segunda.\nY una tercera.'
Patrón: '(^\w+)|(\w+\S*$)'

Tratando la cadena completa como una sola línea:
  ('Este', '')
  ('', 'tercera.')

Considerando las líneas definidas por los saltos de línea:
  ('Este', '')
  ('', 'línea.')
  ('Pero', '')
  ('', 'segunda.')
  ('Y', '')
  ('', 'tercera.')


Normalmente, el metacarácter `.` de una expresión regular encaja con cualquier carácter salvo con el salto de línea (`\n`). El *flag* `re.DOTALL` hace que el metacarácter `.` encaje con cualquier carácter, también con los saltos de línea:

In [27]:
t = 'Este texto ocupa la primera línea.\nPero hay una segunda.\nY una tercera.'
pat = r'.+'

no_newlines = re.compile(pat)
dotall = re.compile(pat, re.DOTALL)

print(f'Texto: {t!r}')
print(f"Patrón: '{pat}'")

print("\nMetacarácter '.' no aceptando saltos de línea:")
for match in no_newlines.findall(t):
    print(f'  {match!r}')

print("\nMetacarácter '.' aceptando saltos de línea (DOTALL):")
for match in dotall.findall(t):
    print(f'  {match!r}')

Texto: 'Este texto ocupa la primera línea.\nPero hay una segunda.\nY una tercera.'
Patrón: '.+'

Metacarácter '.' no aceptando saltos de línea:
  'Este texto ocupa la primera línea.'
  'Pero hay una segunda.'
  'Y una tercera.'

Metacarácter '.' aceptando saltos de línea (DOTALL):
  'Este texto ocupa la primera línea.\nPero hay una segunda.\nY una tercera.'


### Usando una codificación de caracteres distinta a Unicode

En Python 3 la codificación por defecto es `Unicode`, de forma que las expresiones regulares aceptadas por el módulo `re` asumen que tanto el texto como los patrones están codificados en Unicode. Esto significa que, por ejemplo, el patrón `\w+` aceptará tanto la palabra 'compañía' como la palabra 'pingüino', a pesar de que contengan caracteres que no están en la tabla ASCII. El *flag* `re.ASCII` permite restringir la codificación de los caracteres aceptados por los conjuntos predefinidos (como `\w`) a únicamente los de la tabla ASCII. Veamos un ejemplo:

In [28]:
t = 'Excelente compañía ayer en la fiesta del pingüino'
pat = r'\w+'

ascii_pat = re.compile(pat, re.ASCII)
unicode_pat = re.compile(pat)

print(f'Texto: {t!r}')
print(f"Patrón: '{pat}'")

print(f"ASCII: {list(ascii_pat.findall(t))}")
print(f"Unicode: {list(unicode_pat.findall(t))}")

Texto: 'Excelente compañía ayer en la fiesta del pingüino'
Patrón: '\w+'
ASCII: ['Excelente', 'compa', 'a', 'ayer', 'en', 'la', 'fiesta', 'del', 'ping', 'ino']
Unicode: ['Excelente', 'compañía', 'ayer', 'en', 'la', 'fiesta', 'del', 'pingüino']


### Inclusión de comentarios explicativos de los patrones

La forma compacta con que se especifican las expresiones regulares según la sintaxis del módulo `re` puede hacerse casi ilegible a medida que las expresiones se hacen más largas. A medida que se añaden grupos de patrones, resulta más difícil recordar por qué cada grupo está ahí, qué representa y cómo interactúa con el resto de la expresión. Utilizar grupos con nombre ayuda a esclarecer estas situaciones, pero una mejor solución es utilizar el *flag* `re.VERBOSE` en las expresiones, lo que permite añadir comentarios y espacios blancos a la expresión, de manera que sea fácilmente legible.

En el siguiente ejemplo, donde se aplica un patrón para validar direcciones de email pertenecientes a 3 dominios ('.es', '.eus' y '.net'), veremos cómo el modo *verbose* ayuda a entender mejor las expresiones regulares. Empecemos con una versión básica en la que el patrón no está comentado:

In [29]:
email_valido = re.compile('[\w\d.+-]+@([\w\d.]+\.)+(es|eus|net)')

candidatos = ['pablo.matos@a2tv.es',
              'cris.pardo+flequillo@septima.net',
              'pepe-gotera@chapuzas.gv.eus',
              'otilio@otras.chapuzas.eup']

for c in candidatos:
    match = email_valido.search(c)
    print(f"{c:<36s} {'Válido' if match else 'No válido'}")

pablo.matos@a2tv.es                  Válido
cris.pardo+flequillo@septima.net     Válido
pepe-gotera@chapuzas.gv.eus          Válido
otilio@otras.chapuzas.eup            No válido


En su forma extendida (comentada), el patrón realizará la misma tarea pero será más fácil de entender. Los comentarios permitirán identificar las distintas partes del patrón y resultará más fácil modificarlas o ampliarlas.

In [30]:
email_valido = re.compile(
    '''
        [\w\d.+-]+     # Usuario
        @
        ([\w\d.]+\.)+  # Prefijo del dominio
        (es|eus|net)   # Dominios de interés
    ''',
    re.VERBOSE)

candidatos = ['pablo.matos@a2tv.es',
              'cris.pardo+flequillo@septima.net',
              'pepe-gotera@chapuzas.gv.eus',
              'otilio@otras.chapuzas.eup']

for c in candidatos:
    match = email_valido.search(c)
    print(f"{c:<36s} {'Válido' if match else 'No válido'}")

pablo.matos@a2tv.es                  Válido
cris.pardo+flequillo@septima.net     Válido
pepe-gotera@chapuzas.gv.eus          Válido
otilio@otras.chapuzas.eup            No válido


A continuación se presenta una tercera y última versión que acepta entradas que pueden incluir el nombre de una persona seguido de la dirección de email entre los símbolos `<` y `>`, tal como suelen aparecer en las cabeceras de los mensajes. Añadir suficientes comentarios e indentar convenientemente los distintos elementos de los patrones (grupos, subgrupos, etc.) facilita el mantenimiento de los programas, es decir, su comprensión y futura modificación por parte de otros programadores.

In [31]:
email_valido = re.compile(
    '''
        # Un nombre se compone de letras y puede incluir puntos (.)
        # para abreviaturas o iniciales
        ((?P<name>
           ([\w.,]+\s+)*[\w.,]+)
           \s*
           # Las direcciones de email pueden ir entre los símbolos < >
           # pero sólo si se ha suministrado previamente un nombre, 
           # por lo que el símbolo < se incluye dentro de este grupo
           <
        )? # El nombre es opcional
        
        # Aquí va la dirección de email
        (?P<email>
          [\w\d.+-]+     # Usuario
          @
          ([\w\d.]+\.)+  # Prefijo del dominio
          (es|eus|net)   # Dominios de interés
        )

        >? # El símbolo > es opcional
    ''',
    re.VERBOSE)

candidatos = ['pablo.matos@a2tv.es',
              'cris.pardo+flequillo@septima.net',
              'pepe-gotera@chapuzas.gv.eus',
              'otilio@otras.chapuzas.eup',
              'Bob Pop <bobby.pop@comic.net>',
              'Nancy Rubia nancy.blonde@a2tv.es',
              'Lehendakari Urkullu',
              'Francisco F. Capela <patxi.capela@etbb.eus>',
              '<nacho.canut@fangorio.es>']

for c in candidatos:
    print(f'\nCandidato: {c!r}')
    match = email_valido.search(c)
    if match:
        print(f"  Nombre: {match.groupdict()['name']!r}")
        print(f"  Email:  {match.groupdict()['email']!r}")
    else:
        print('  No match')


Candidato: 'pablo.matos@a2tv.es'
  Nombre: None
  Email:  'pablo.matos@a2tv.es'

Candidato: 'cris.pardo+flequillo@septima.net'
  Nombre: None
  Email:  'cris.pardo+flequillo@septima.net'

Candidato: 'pepe-gotera@chapuzas.gv.eus'
  Nombre: None
  Email:  'pepe-gotera@chapuzas.gv.eus'

Candidato: 'otilio@otras.chapuzas.eup'
  No match

Candidato: 'Bob Pop <bobby.pop@comic.net>'
  Nombre: 'Bob Pop'
  Email:  'bobby.pop@comic.net'

Candidato: 'Nancy Rubia nancy.blonde@a2tv.es'
  Nombre: None
  Email:  'nancy.blonde@a2tv.es'

Candidato: 'Lehendakari Urkullu'
  No match

Candidato: 'Francisco F. Capela <patxi.capela@etbb.eus>'
  Nombre: 'Francisco F. Capela'
  Email:  'patxi.capela@etbb.eus'

Candidato: '<nacho.canut@fangorio.es>'
  Nombre: None
  Email:  'nacho.canut@fangorio.es'


### Patrones con flags

En ocasiones, no es posible incluir *flags* al compilar una expresión regular. Esto sucede, por ejemplo, al pasar un patrón como argumento a una función que lo compilará más tarde. En estos casos, el *flag* puede incluirse en el propio patrón. Estos *flags* deben ir al principio de la expresión para que afecten a todos los elementos de dicha expresión. Así, por ejemplo, añadiendo la cadena `(?i)` al inicio de la expresión regular, estamos indicando que no se tenga en cuenta el tamaño (mayúscula/minúscula) de las letras:

In [32]:
t = 'Esto, por exigencias de guión, es un texto con puntuación.'
pat = r'(?i)\bE\w+'

regex = re.compile(pat)
print(f'Texto: {t!r}')
print(f"Patrón: '{pat}'")
print(f'Instancias encontradas: {regex.findall(t)}')

Texto: 'Esto, por exigencias de guión, es un texto con puntuación.'
Patrón: '(?i)\bE\w+'
Instancias encontradas: ['Esto', 'exigencias', 'es']


Se pueden especificar varios flags al mismo tiempo; por ejemplo, la cadena `(?im)` hace que no se tenga en cuenta el tamaño de las letras y que se procesen cadenas multilínea. En la tabla siguiente se muestran todas las abreviaturas junto con los *flags* correspondientes:

Abreviatura | Flag
:-- | :--
a | ASCII
i | IGNORECASE
m | MULTILINE
s | DOTALL
x | VERBOSE